# Modeling

## Load libraries and utility functions.

In [1]:
from __future__ import print_function
import os
import warnings
import pandas as pd
import lightgbm as lgb
from sklearn.feature_extraction import text
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.externals import joblib
from ItemSelector import ItemSelector
from label_rank import label_rank

warnings.filterwarnings(action='ignore', category=UserWarning,
                        module='lightgbm')

## Define the input parameters.
One of the most important parameters is the number of estimators that allows you to trade-off accuracy and modeling time. The table below should give you an idea of the relationships between the number of estimators and the metrics.

| estimators | run time (s) | Accuracy@1 | Accuracy@2 | Accuracy@3 |
|------------|--------------|----------|--------|--------|
|        100 |           40 |   25.02% | 38.72% | 47.83% |
|       1000 |          177 |   46.79% | 60.80% | 69.11% |
|       2000 |          359 |   51.38% | 65.93% | 73.09% |
|       4000 |          628 |   53.39% | 67.40% | 74.74% |

In [2]:
args_data = 'balanced_pairs_train.tsv'
args_test = 'balanced_pairs_test.tsv'
args_estimators = 4000
args_ngrams = 1
args_unweighted = False
args_min_child_samples = 20
args_match = 20
args_outputs = '.'
args_inputs = '.'
args_save = True
args_model = 'model.pkl'
args_instances = 'inst.txt'
args_labels = 'labels.txt'
args_rank = 3
args_verbose = -1

## Define the paths to the input and output data

In [3]:
# The input data.
inputs_path = args_inputs
data_path = os.path.join(inputs_path, args_data)
test_path = os.path.join(inputs_path, args_test)

# The output data.
outputs_path = args_outputs
model_path = os.path.join(outputs_path, args_model)
instances_path = os.path.join(outputs_path, args_instances)
labels_path = os.path.join(outputs_path, args_labels)

# Create the outputs folder.
os.makedirs(outputs_path, exist_ok=True)

## Load and set up the training data

In [4]:
# Load the training data.
print('Reading {}'.format(data_path))
train = pd.read_csv(data_path, sep='\t', encoding='latin1')
train

Reading ./balanced_pairs_train.tsv


,Id_x,AnswerId_x,Text_x,Id_y,Text_y,AnswerId_y,Label,n
0,114525,336868,"the difference between the two functions? (""fu...",336859,var functionname = function() {} vs function f...,336868,1,0
1,114525,336868,"the difference between the two functions? (""fu...",1566595,can i use multiple versions of jquery on the s...,1566644,0,1
2,114525,336868,"the difference between the two functions? (""fu...",32584850,facebook js sdk's fb.api('/me') method doesn't...,32585470,0,2
3,114525,336868,"the difference between the two functions? (""fu...",1359018,"in jquery, how to attach events to dynamic htm...",9331127,0,3
4,114525,336868,"the difference between the two functions? (""fu...",2655925,how to apply !important using .css()?. i am ha...,8894528,0,4
5,114525,336868,"the difference between the two functions? (""fu...",20279484,how to access the correct `this` / context ins...,20279485,0,5
6,114525,336868,"the difference between the two functions? (""fu...",1144783,replacing all occurrences of a string in javas...,17606289,0,6
7,114525,336868,"the difference between the two functions? (""fu...",1398582,prevent execution of parent event handler. i h...,1398608,0,7
8,114525,336868,"the difference between the two functions? (""fu...",391979,get client ip using just javascript?. i need t...,810461,0,8
9,114525,336868,"the difference between the two functions? (""fu...",5223,"length of a javascript object (that is, associ...",6700,0,9


In [5]:
# Limit the number of training duplicate matches.
train = train[train.n < args_match]

# The input data columns.
feature_columns = ['Text_x', 'Text_y']
label_column = 'Label'
group_column = 'Id_x'
answerid_column = 'AnswerId_y'
name_columns = ['Id_x', 'Id_y']

# Report on the dataset.
print('train: {:,} rows with {:.2%} matches'.format(
    train.shape[0], train[label_column].mean()))

train: 132,500 rows with 5.00% matches


In [6]:
# Compute instance weights.
weight_column = 'Weight'
if args_unweighted:
    weight = pd.Series([1.0], train[label_column].unique())
else:
    label_counts = train[label_column].value_counts()
    weight = train.shape[0]/(label_counts.shape[0]*label_counts)
train[weight_column] = train[label_column].apply(lambda x: weight[x])

# Collect the ordered AnswerId.
labels = sorted(train[answerid_column].unique())
label_order = pd.DataFrame({'label': labels})

## Define the model.

In [7]:
# Select and format the training data.
train_X = train[feature_columns]
train_y = train[label_column]
sample_weight = train[weight_column]
names = train[name_columns]

In [8]:
# Select the training hyperparameters.
n_estimators = args_estimators
min_child_samples = args_min_child_samples
estimator = lgb.LGBMClassifier(n_estimators=n_estimators,
                               min_child_samples=min_child_samples,
                               verbose=args_verbose)
if args_ngrams > 0:
    ngram_range = (1, args_ngrams)
else:
    ngram_range = None
assert ngram_range is not None

# The featurization pipeline(s) for each text column.
featurization = [
    (column,
     make_pipeline(ItemSelector(column),
                   text.TfidfVectorizer(ngram_range=ngram_range)))
    for column in feature_columns]
features = FeatureUnion(featurization)

# The model pipeline.
model = Pipeline([
    ('features', features),
    ('model', lgb.LGBMClassifier(n_estimators=n_estimators))
])

## Fit the model.

In [9]:
%%time
model.fit(train_X, train_y, model__sample_weight=sample_weight)

CPU times: user 21min 33s, sys: 2.15 s, total: 21min 35s
Wall time: 3min 59s


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('Text_x', Pipeline(memory=None,
     steps=[('itemselector', ItemSelector(keys='Text_x')), ('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf...0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0))])

Write the model to file.

In [10]:
if args_save:
    joblib.dump(model, model_path)

## Test the model

In [11]:
# Read the test data.
print('Reading {}'.format(test_path))
test = pd.read_csv(test_path, sep='\t', encoding='latin1')
print('test {:,} rows with {:.2%} matches'.format(
    test.shape[0], test[label_column].mean()))

Reading ./balanced_pairs_test.tsv
test 297,570 rows with 0.55% matches


In [12]:
test.head()

,Id_x,AnswerId_x,Text_x,Id_y,Text_y,AnswerId_y,Label,n
0,392561,69984,firefox script tag error. while adding some ve...,69913,why don't self-closing script tags work?. what...,69984,1,0
1,392561,69984,firefox script tag error. while adding some ve...,2194992,jquery - $ is not defined. i have a simple jqu...,2195167,0,1
2,392561,69984,firefox script tag error. while adding some ve...,13418669,javascript: do i need to put this.var for ever...,13418980,0,2
3,392561,69984,firefox script tag error. while adding some ve...,2421911,what is the purpose of wrapping whole javascri...,2421949,0,3
4,392561,69984,firefox script tag error. while adding some ve...,1771786,question mark in javascript. i came across the...,1771824,0,4


Collect the model predictions.

In [13]:
%%time
test_X = test[feature_columns]
test['probabilities'] = model.predict_proba(test_X)[:, 1]

CPU times: user 2min 16s, sys: 540 ms, total: 2min 16s
Wall time: 52.5 s


In [14]:
test.head()

,Id_x,AnswerId_x,Text_x,Id_y,Text_y,AnswerId_y,Label,n,probabilities
0,392561,69984,firefox script tag error. while adding some ve...,69913,why don't self-closing script tags work?. what...,69984,1,0,0.978580
1,392561,69984,firefox script tag error. while adding some ve...,2194992,jquery - $ is not defined. i have a simple jqu...,2195167,0,1,0.007486
2,392561,69984,firefox script tag error. while adding some ve...,13418669,javascript: do i need to put this.var for ever...,13418980,0,2,0.000124
3,392561,69984,firefox script tag error. while adding some ve...,2421911,what is the purpose of wrapping whole javascri...,2421949,0,3,0.000021
4,392561,69984,firefox script tag error. while adding some ve...,1771786,question mark in javascript. i came across the...,1771824,0,4,0.000503


Create a data frame with one row per duplicate question, and make it contain the model's predictions.

In [15]:
# Order the testing data by dupe Id and question AnswerId.
test.sort_values([group_column, answerid_column], inplace=True)

# Extract the ordered probabilities.
probabilities = (
    test.probabilities
    .groupby(test[group_column], sort=False)
    .apply(lambda x: tuple(x.values)))

In [16]:
# Get the individual records.
output_columns_x = ['Id_x', 'AnswerId_x', 'Text_x']
test_score = (test[output_columns_x]
              .drop_duplicates()
              .set_index(group_column))
test_score['probabilities'] = probabilities
test_score.reset_index(inplace=True)
test_score.columns = ['Id', 'AnswerId', 'Text', 'probabilities']

In [17]:
test_score

,Id,AnswerId,Text,probabilities
0,392561,69984,firefox script tag error. while adding some ve...,"(3.4876173639007326e-06, 0.0533047302100469, 1..."
1,440494,169035,share constants between php and javascript. p...,"(0.00047325394589784565, 0.0014949612856234923..."
2,562412,14220323,return value from function with an ajax call. ...,"(2.1791113431862293e-06, 2.96331313732673e-05,..."
3,896449,805113,what is the best way to have long string liter...,"(0.0011363680653459676, 0.0024452270511267477,..."
4,933343,750506,how are local variables referenced in closures...,"(9.562679115882025e-06, 1.6924632753746486e-05..."
5,1396307,1433217,javascript replace function won't remove the s...,"(1.74912473223069e-06, 5.1880700336476486e-05,..."
6,1530423,1433217,javascript replace doesn't work when it should...,"(6.516775540273877e-05, 0.0004557502711386043,..."
7,1582634,750506,passing values to onclick. if i create a whole...,"(0.0001007688602192682, 0.0001871309607861537,..."
8,1689679,1129270,javascript: how to sort an array of records by...,"(0.0004074041040203017, 5.7548990733810616e-05..."
9,1723287,359910,calling a javascript function named in a varia...,"(5.987516761591326e-06, 2.9187152425722594e-05..."


## Evaluate the predictions

In [18]:
# Rank the correct answers.
test_score['Ranks'] = test_score.apply(lambda x:
                                       label_rank(x.AnswerId,
                                                  x.probabilities,
                                                  label_order.label),
                                       axis=1)

# Compute the number of correctly ranked answers
for i in range(1, args_rank+1):
    print('Accuracy @{} = {:.2%}'.format(
        i, (test_score['Ranks'] <= i).mean()))
mean_rank = test_score['Ranks'].mean()
print('Mean Rank {:.4f}'.format(mean_rank))

# Write the scored instances.
test_score.to_csv(instances_path, sep='\t', index=False,
                  encoding='latin1')
label_order.to_csv(labels_path, sep='\t', index=False)

Accuracy @1 = 53.46%
Accuracy @2 = 66.91%
Accuracy @3 = 74.01%
Mean Rank 5.3211
